# Time Series Machine Learning Part 1 Assignment

In [0]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

### Import the Netflix stock price data set (NFLX_data.csv).

In [0]:
df=pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Time%20Series%20Analysis/Time%20Series%20-%20Day%205/NFLX_data.csv')

In [28]:
df.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,25.9635,26.2800,25.7157,25.8528,25649820,NFLX
1,2013-02-11,25.5685,26.0071,24.9714,25.4128,29321782,NFLX
2,2013-02-12,25.8085,26.2228,25.1014,25.4214,34388802,NFLX
3,2013-02-13,25.8428,26.6285,25.6657,26.6098,40799094,NFLX
4,2013-02-14,26.7557,27.1214,26.3844,26.7714,31968685,NFLX


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 7 columns):
date      1259 non-null object
open      1259 non-null float64
high      1259 non-null float64
low       1259 non-null float64
close     1259 non-null float64
volume    1259 non-null int64
Name      1259 non-null object
dtypes: float64(4), int64(1), object(2)
memory usage: 69.0+ KB


In [0]:
df['date'] = pd.to_datetime(df['date'])

### Transform the data by shifting the series and creating features that will allow us to forecast the price 30 days into the future from 90 days of daily history.

In [0]:
history = 90
future = 30

shifts = [x + future for x in list(range(1,history+1))]
for shift in shifts:
    df['t-' + str(shift)] = df['close'].shift(shift)

df.dropna(inplace=True)

### Split the data into a training set and a testing set. Make the test set size 20%.

In [0]:
idx =int(len(df) * 0.8)

In [0]:
train =df.iloc[:idx]
test = df.iloc[idx:]

In [0]:
test.head()

In [0]:
test['close'].iloc[100:]

In [0]:
def ts_split(x,y,test_size=0.2):
    idx =int(len(x) * (1-test_size))
    return x.iloc[:idx], x.iloc[idx:], y.iloc[:idx], y.iloc[idx:]

In [0]:
ts_split(X,y)

In [0]:
X = df.drop(['date','close','Name'],axis=1)
y= df['close']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

### Instantiate an AdaBoost model and fit it to the training set.

In [0]:
model = AdaBoostRegressor()
ada = model.fit(X_train,y_train)

### Generate predictions for the test set.

In [0]:
predictions = model.predict(X_test)

### Evaluate the results using R-Squared, Mean Absolute Error, and Root Mean Squared Error metrics.

In [55]:
print('R-squared:',r2_score(y_test,predictions))
print('Mean Absolute Error:',mean_absolute_error(y_test,predictions))
print('Root Mean Squared Error:',np.sqrt(mean_squared_error(y_test,predictions)))

R-squared: 0.9924170771722459
Mean Absolute Error: 3.014808462350679
Root Mean Squared Error: 3.8830176555136315


### Visually examine the results by creating a scatter plot where the x axis represents the observed results and the y axis represents the predictions.

In [0]:
def iscatter(df,x,y,color=None,title=''):
    fig = px.scatter(df, x=x,y=y, color=color,size=None,title=title, template='none')
    fig.update_traces(marker_line_color='black',marker_line_width=1)
    fig.show();

In [57]:
iscatter(df,x=y_test,y=predictions, title="Close v. Predictions")